# <span style="color:Maroon">ПРОЕКТ: Оптимизация производственных расходов</span>

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Чтобы оптимизировать производственные расходы, металлургический комбинат решил уменьшить потребление электроэнергии на этапе обработки стали.<br><br>    
<b>Цель данного проекта</b> - Необходимо построить модель, которая предскажет температуру стали<br><br>    
Заказчику важны:<br>
<li>Модель со значением метрики качества <b>MAE</b> не более <b>8.7</b>.<br><br>    
<span style="color:Maroon"><b>1 Загрузить и подготовить данные:</b></span><br><br>
<li>Проверить состав предоставленной выбороки;
<li>Проанализировать данные;
<li>Провести предобработку данных;
<li>Описать результаты.<br><br>
<span style="color:Maroon"><b>2 Обучить модель и выбрать лучшую:</b></span><br><br>
<li>Обучите разные модели на базовых настройках;
<li>Обучите разные модели на с подбором гиперпараметров;
<li>Подобрать для моделей оптимальные гиперпараметры;
<li>Оценить качество моделей кросс-валидацией;
<li>Выбрать лучшую модель
<li>Сделать выводы.<br><br>
<span style="color:Maroon"><b>3 Тестирование лучшей модели:</b></span><br><br>
<li>Проверить модель на тестовой выборке;
<li>Проанализировать предсказания выбранной модели;
<li>Написать выводы и обосновать выбор.
 </div>

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
создам стиль для отображения табличных данных
</div>

In [ ]:
cell_hover     = {'selector': 'td:hover',
                  'props'   : [('background', '#9E4447'), 
                               ('color', '#ffffff')]}           # формат выделенной ячейки

row_hover      = {'selector': 'tr:hover',
                  'props'   : [('background', '#808080'), 
                               ('color', '#ffffff')]}           # формат выделенной строки

color_row_even = {'selector': 'tr:nth-of-type(even)',
                  'props'   : [('background', '#D9D9D9'),
                               ('color', 'black')]}             # формат нечетных строк

color_row_odd  = {'selector': 'tr:nth-of-type(odd)',
                  'props'   : [('background', '#ffffff'),
                               ('color', '#363636')]}           # формат четных строк

index_names    = {'selector': 'th',
                  'props'   : [('background', '#363636'), 
                               ('color', '#ffffff'),  
                               ('text-align','center')]}        # формат заголовка и индекса

border_inner   = {'selector': 'td',
                  'props'   : [('border','1px dashed #363636')]}# формат границы таблицы

border_outer   = {'selector': '',
                  'props'   : [('border','2px solid #363636')]} # формат границы таблицы

caption        = {'selector': 'caption',
                  'props'   : [('color', '#363636'), 
                               ('font-size', '15px')]}

# передаю в переменную для дальнейшего использования
styler = [cell_hover, color_row_even, color_row_odd, index_names, row_hover, border_inner, border_outer, caption]

## Загрузка и подготовка данных

In [ ]:
# библиотеки
import re
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings 
from matplotlib import pyplot as plt
from time import time

# обработка
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# модели
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor

# метрики
from sklearn.metrics import mean_absolute_error

# настройки и параметры
from tqdm import notebook
RANDOM     = 1123581321
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [ ]:
data_arc       = pd.read_csv('data_arc.csv')
data_bulk      = pd.read_csv('data_bulk.csv')
data_bulk_time = pd.read_csv('data_bulk_time.csv')
data_gas       = pd.read_csv('data_gas.csv')
data_temp      = pd.read_csv('data_temp.csv')
data_wire      = pd.read_csv('data_wire.csv')
data_wire_time = pd.read_csv('data_wire_time.csv')

In [ ]:
data_flow      = [data_arc, data_bulk, data_bulk_time, data_gas, data_temp, data_wire, data_wire_time]
data_flow_name = ['data_arc', 'data_bulk', 'data_bulk_time', 'data_gas', 'data_temp', 'data_wire', 'data_wire_time']

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Посмотрим на качество предоставленных данных
</div>

In [ ]:
for name, data in enumerate(data_flow):
    print(f'Набор данных {data_flow_name[name]}')
    print()
    display(data.info())
    print('='*100)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
После первого осмотра данных выявлены следующие замечания:<br>
<li>основной датасет <b>data_temp</b>, в нем находится информация по температуре на момент завершения технологического процесса плавки металла, последняя запись о замере температуры в партии является целевым признаком. Соответственно построение финального датасета будет ориентированно именно на этот набор данных;
<li>В датасете <b>data_temp</b> есть большое количество пропусков относительно общего набора данных, восстанавливать эти данные нет смысла и может только навредить моделированию, т.к. целевой ориентир может быть искажен;
<li>Общие замечание по наборам данных: наличие большого количества пропусков в данных, название столбцов не в "питонском формате", тип данных <b><i>object</i></b> где информация о времени, необходимо изменить тип.
<li>Далее пройдемся по каждому набору данных более детально, но предварительно сразу изменю название столбцов и поменяю тип данных.
</div>

In [ ]:
data_arc.columns  = ['key', 'time_start_heat', 'time_end_heat', 'active_pwr', 'reactive_pwr']
data_gas.columns  = ['key', 'gas']
data_temp.columns = ['key', 'time_measure', 'temperature']

In [ ]:
# функция для изменения формата даты
def object_to_date(data):
    for column in data.columns:
        if data[column].dtype == 'object':
            data[column] = pd.to_datetime(data[column], format="%Y-%m-%d %H:%M:%S")
    return data

In [ ]:
# функция для переименования столбцов в "питонский формат"
def columns_rename(data):
    columns_new = []
    for column in data.columns:
        columns_new.append(column.replace(' ', '_').lower())
    data.columns = columns_new
    return data

In [ ]:
for data in data_flow:
    columns_rename(data)
    object_to_date(data)  

In [ ]:
for name, data in enumerate(data_flow):
    display(data.head(10).style\
                         .set_caption(f'Набор данных {data_flow_name[name]}')\
                         .set_table_styles(styler))
    display(data.info())
    print('='*100)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Первые изменения внесены, теперь необходимо познакомиться с составом информации по партиям в каждом наборе данных, чтобы понять насколько большие различия в них, т.к. некоторые таблицы содержат уже агрегированные данные для каждой партии, а некоторые несут дополнительную информацию по итерациям нагрева, необходимо понять сколько всего уникальных партий
</div>

In [ ]:
for name, data in enumerate(data_flow):
    print(f'Количество партий {len(data["key"].unique())} в наборе данных {data_flow_name[name]}')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Как видно из информации выше, в каждом датасете имеется разное количество партий, за исключение только информации о материалах, но это очевидно т.к. они сильно зависимы. Это очень странная ситуация, которая говорит о том, что выгрузка была произведена скорее всего не верно, качество предоставленных данных оставляет желать лучшего. Здесь бы я, наверное, обратился к тому, кто выгружал их, но за неимением такой возможности продолжим работать с тем что есть.
</div>

## Анализ данных

### Анализ данных по замерам температуры

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Первый датасет для анализа выбран не случайно, как я говорил выше это основной набор информации, на котором будет строится аналитика. В этом датасете мне необходимо получить информацию для каждой партии о первом замере температуры и последнем, для этого промаркирую набор данных
</div>

In [ ]:
target = data_temp.copy()

In [ ]:
target.loc[target.sort_values("time_measure").
       groupby("key").
       apply(lambda x: x.index[0]), "mark"] = 'first'

In [ ]:
target.loc[target.sort_values("time_measure", ascending = False).
       groupby("key").
       apply(lambda x: x.index[0]), "mark"] = 'last'

In [ ]:
target.head().style.format({'temperature':'{:.0f}'}).set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Необходимые данные помечены, посмотрим на соотношение информации и проверим есть ли пары для каждой записи
</div>

In [ ]:
target.query('mark == "last"').shape[0], target.query('mark == "first"').shape[0]

In [ ]:
data_temp.value_counts('key').sort_values(ascending = True).head()

In [ ]:
target.info()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Как можно заметить есть две партии для которых существует только одна запись с замером температуры, возможно это правда, но у меня большие сомнения по поводу этой ситуации, поэтому мне придется удалить эти партии, считаю их не корректными. Так же после объединения данных избавлюсь от пропусков
</div>

In [ ]:
target = target.query('mark == "last"')[['key', 'time_measure', 'temperature']]\
               .merge(target.query('mark == "first"')[['key', 'time_measure', 'temperature']],
                      on = 'key', 
                      how = 'left',
                      suffixes = ("_finish", "_start"))\
               .rename(columns = {'time_measure_finish' : 'time_finish_measure', 'time_measure_start' : 'time_start_measure'})

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Очистим оставшиеся данные от возможных пропусков и также проведем проверку датасета на предмет наличия данных в которых начальная температура равна финальной, и удалим эти данные из за возможной утечки признака
</div>

In [ ]:
target = target.dropna()
target = target.query('temperature_start !=  temperature_finish')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Посмотрим на распределение данных, возможно обнаружим аномалии или выбросы
</div>

In [ ]:
plt.figure(figsize = (15, 7))
sns.set_style('whitegrid')
x = ['temperature_start', 'temperature_finish']

for i, temp in enumerate(x):
    
    plt.subplot(1, 2, i + 1)
    plt.title(x[i], fontsize=16)
    sns.set_style('whitegrid')
    sns.violinplot(data = target[temp],
                   color      = '#c0504d',
                   saturation = 0.6,
                   linewidth  = 3,
                   edgecolor  = '#363636')
plt.show()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Как видно из графиков распределение не равномерное есть некоторые аномалии особенно что касается температуры старта, необходимо посмотреть если количество небольшое, можно принять за нижний уровень температуру 1500. С показателями температуры финиша, все сложнее особенно в ее верхней части, где после небольшого провала появляются данные. Если ориентироваться на данные по начальной температуре, то диапазон выглядит корректным, попробую оставить значения без изменений.
</div>

In [ ]:
target.query('temperature_start < 1500')\
      .style\
      .format({'temperature_finish':'{:.0f}', 'temperature_start':'{:.0f}'})\
      .set_table_styles(styler)

In [ ]:
target = target.query('temperature_start > 1500')

In [ ]:
plt.figure(figsize = (15, 7))
sns.set_style('whitegrid')
x = ['temperature_start', 'temperature_finish']

for i, temp in enumerate(x):
    
    plt.subplot(1, 2, i + 1)
    plt.title(x[i], fontsize=16)
    sns.set_style('whitegrid')
    sns.violinplot(data = target[temp],
                   palette    = 'bone',
                   saturation = 0.6,
                   linewidth  = 3,
                   edgecolor  = '#363636')
plt.show()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Теперь набор данных выглядит нормально распределенным, конечно сильное искажение в данных по температуре последнего замера, но я оставлю его как есть. Посмотрим на объем утерянных данных
</div>

In [ ]:
target.shape[0], len(data_temp["key"].unique())

In [ ]:
'{:.0%}'.format(1 - target.shape[0] / len(data_temp["key"].unique()))

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Серьёзные изменения в 24% от первоначального состава, я бы вернул эти данные на доработку или проверку выгрузки
</div>

In [ ]:
target.head().style.format({'temperature_finish':'{:.0f}', 'temperature_start':'{:.0f}'}).set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Базовый набор данных определен, 2429 записей, т.е. партий на начальном этапе, данные по времени и начальной температуре сохранены для дальнейшего использования на стадии определения признаков и проверки корректности данных
</div>

### Анализ данных сыпучих материалов

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Далее проверим данные по материалам из наборов <b>data_bulk</b> и <b>data_bulk_time</b>. Как и на предыдущем шаге, здесь необходимо получить информацию по начальному и конечному времени засыпки материалов в ковш, для этого построчно проверю и найду максимальное и минимальное значение времени. Большое количество пропусков связано с тем, что не во всех партиях используется материалы некоторых видов, поэтому заполнять пропуски я буду заглушкой из 0. Проверю так же чтобы не было записей совсем пустых
</div>

In [ ]:
bulk_time = data_bulk_time.copy()

In [ ]:
bulk_time = bulk_time.assign(time_start_bulk  = (bulk_time.loc[:, 'bulk_1':'bulk_15']).min(axis = 1),
                             time_finish_bulk = (bulk_time.loc[:, 'bulk_1':'bulk_15']).max(axis = 1))

In [ ]:
(data_bulk.loc[:, 'bulk_1':'bulk_15']).sum(axis = 1).isna().sum()

In [ ]:
bulk_time.head().style.set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Информация добавлена, так же проверены пустые строки - таких нет. Посмотрим на распределение каждого материала
</div>

In [ ]:
plt.figure(figsize = (13, 15))
sns.set_style('whitegrid')
x = list(data_bulk.columns[1:])

for i, bulk in enumerate(x):
    
    plt.subplot(5, 3, i + 1)
    sns.histplot(data = data_bulk[bulk],
                 color = '#9E4447',
                 element="step",
                 bins = 100)
    
    plt.title(f'{x[i]}', fontsize = 14)
    plt.ylabel('spread', fontsize = 14)
    plt.xlabel('')
    plt.tight_layout()    
plt.show()

In [ ]:
for i in list(data_bulk.columns[1:]):
    print(f'записей в признаке {i} = {data_bulk[i].count()}')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Данные очень сильно неравномерны, сильный дисбаланс в классах, некоторые признаки имею только одну запись <b>bulk_8</b>, что возможно на финальном этапе необходимо будет удалить те признаки, у которых менее 100 записей т.к. никакого влияния не будет нести, очень мало данных и сильный разброс, что опять говорить о низком качестве предоставленной информации
</div>

### Анализ данных проволочных материалов

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Далее проверим данные по материалам из наборов <b>data_wire</b> и <b>data_wire_time</b>. Как и на предыдущем шаге, здесь необходимо получить информацию по начальному и конечному времени засыпки материалов в ковш, для этого построчно проверю и найду максимальное и минимальное значение времени. Большое количество пропусков связано с тем, что не во всех партиях используется материалы некоторых видов, поэтому заполнять пропуски я буду заглушкой из 0. Проверю так же чтобы не было записей совсем пустых
</div>

In [ ]:
wire_time = data_wire_time.copy()

In [ ]:
wire_time = wire_time.assign(time_start_wire  = (wire_time.loc[:, 'wire_1':'wire_9']).min(axis = 1),
                             time_finish_wire = (wire_time.loc[:, 'wire_1':'wire_9']).max(axis = 1))

In [ ]:
(data_wire.loc[:, 'wire_1':'wire_9']).sum(axis = 1).isna().sum()

In [ ]:
wire_time.head().style.set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Информация добавлена, так же проверены пустые строки - таких нет. Посмотрим на распределение каждого материала
</div>

In [ ]:
plt.figure(figsize = (13, 11))
sns.set_style('whitegrid')
x = list(data_wire.columns[1:])

for i, wire in enumerate(x):
    
    plt.subplot(3, 3, i + 1)
    sns.histplot(data = data_wire[wire],
                 color = '#444444',
                 element="step",
                 bins = 100)
    
    plt.title(f'{x[i]}', fontsize = 16)
    plt.ylabel('spread', fontsize = 14)
    plt.xlabel('')
    plt.tight_layout()    
plt.show()

In [ ]:
for i in list(data_wire.columns[1:]):
    print(f'записей в признаке {i} = {data_wire[i].count()}')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Данные очень сильно неравномерны, сильный дисбаланс в классах, некоторые признаки имею только одну запись <b>wire_5</b>, что возможно на финальном этапе необходимо будет удалить, т.к. 90% набора это всего лишь два признака, те признаки у которых менее 100 записей т.к. никакого влияния не будет нести, очень мало данных и сильный разброс, что опять говорить о низком качестве предоставленной информации
</div>

### Анализ данных электродов

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Данный датасет интересен тем, что в нем информация по технологическому процессу нагрева металла, где угазаны периоды нагрева, а также данные по затраченной энергии. Этот датасет очень важен для подготовки данных из него я возьму информацию о времени начала техпроцесса о времени его завершения, на основании этой информации я выведу новый показатель о продолжительности техпроцесса для каждой партии в агрегированном состоянии. Так же т.к. мы имеем информацию о активной и реактивной мощности, то я найду полную потраченную мощность по формуле и создам на основании нее новый показатель. Время мне поможет найти, если такие имеются данные которые противоречат технологическому процессу. Я буду сравнивать время засыпки материалов и замера температуры именно по этим данным чтобы точно отсечь аномалии
</div>

In [ ]:
heat_data = data_arc.copy()

In [ ]:
heat_data = heat_data.assign(total_time = heat_data['time_end_heat'] - heat_data['time_start_heat'])

In [ ]:
heat_data['total_time'] = heat_data['total_time'].astype('timedelta64[s]')

In [ ]:
heat_data['total_power'] = np.sqrt(np.power(heat_data['active_pwr'],2) + np.power(heat_data['reactive_pwr'],2))

In [ ]:
heat_data.loc[heat_data.sort_values("time_start_heat").
       groupby("key").
       apply(lambda x: x.index[0]), "mark"] = 'first'

In [ ]:
heat_data.loc[heat_data.sort_values("time_end_heat", ascending = False).
       groupby("key").
       apply(lambda x: x.index[0]), "mark"] = 'last'

In [ ]:
heat_data.head().style.set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Данные размечены, до того, как буду выводить агрегированные данные необходимо ознакомиться с распределением информации по мощности на предмет выбросов и аномалий
</div>

In [ ]:
plt.figure(figsize = (15, 7))
sns.set_style('whitegrid')
x = ['active_pwr', 'reactive_pwr']

for i, power in enumerate(x):
    
    plt.subplot(1, 2, i + 1)
    plt.title(x[i], fontsize=16)
    sns.set_style('whitegrid')
    sns.violinplot(data = heat_data[power],
                   color      = '#c0504d',
                   saturation = 0.6,
                   linewidth  = 3,
                   edgecolor  = '#363636')
plt.show()

In [ ]:
heat_data.describe().style.format('{:.2f}').set_table_styles(styler)

In [ ]:
heat_data.query('reactive_pwr < 0').style.set_table_styles(styler)

In [ ]:
heat_data.query('active_pwr > 3').style.set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Как видно из графика есть заметные выбросы или даже аномалии, так для реактивной мощности есть явно аномальное значение -715,5, его необходимо удалить. Для набора данных по активной мощности тоже есть выбросы со значение более 3, попробую их оставить возможно они не сильно повлияют на конечный результат
</div>

In [ ]:
heat_data = heat_data.query('key != 2116')

In [ ]:
plt.figure(figsize = (15, 7))
sns.set_style('whitegrid')
x = ['active_pwr', 'reactive_pwr']

for i, power in enumerate(x):
    
    plt.subplot(1, 2, i + 1)
    plt.title(x[i], fontsize=16)
    sns.set_style('whitegrid')
    sns.violinplot(data = heat_data[power],
                   palette    = 'bone',
                   saturation = 0.6,
                   linewidth  = 3,
                   edgecolor  = '#363636')
plt.show()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Теперь распределения выглядят симметрично
</div>

In [ ]:
data_temp.value_counts('key').sort_values(ascending = True).head()

In [ ]:
heat_data.query('mark == "first"')[['key', 'time_start_heat']]\
         .merge(heat_data.query('mark == "last"')[['key', 'time_end_heat']],
                on = 'key', 
                how = 'outer').isna().sum()

In [ ]:
heat_data.groupby('key')['total_time', 'total_power'].sum().isna().sum()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Проверил сколько пар отсутствует по времени для начала и финиша, всего две, также предварительно посмотрел сколько будет пропусков в результате объединения и из-за того, что данные с пропусками. Потери будут не критичными можно объединить данные
</div>

In [ ]:
heat_data = heat_data.query('mark == "first"')[['key', 'time_start_heat']]\
                     .merge(heat_data.query('mark == "last"')[['key', 'time_end_heat']],
                            on = 'key', 
                            how = 'outer')\
                     .merge(heat_data.groupby('key')['total_time', 'total_power'].sum(),
                            on = 'key', 
                            how = 'outer')

In [ ]:
heat_data.head()\
         .style\
         .set_caption(f'Набор данных для электродов')\
         .format({'total_time' : '{:.0f}', 'total_power' : '{:.2f}'})\
         .set_table_styles(styler)

In [ ]:
heat_data.info()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Таблица подготовлена, есть небольшое количество пропусков, оставлю пока данные в таком виде при финальном объединении датасетов можно будет удалить
</div>

### Свод данных

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Теперь можно приступить к объединению подготовленных данных в один датасет, объединять буду по ключу <b>key</b>, на основании датасета <b>data_temp/target</b>, так как в нем есть целевой показатель 
</div>

In [ ]:
raw_data = target.merge(bulk_time[['key', 'time_start_bulk', 'time_finish_bulk']], on = 'key', how = 'left')\
                 .merge(data_bulk.fillna(0), on = 'key', how = 'left')\
                 .merge(wire_time[['key', 'time_start_wire', 'time_finish_wire']], on = 'key', how = 'left')\
                 .merge(data_wire.fillna(0), on = 'key', how = 'left')\
                 .merge(data_gas, on = 'key', how = 'left')\
                 .merge(heat_data, on = 'key', how = 'left')

In [ ]:
raw_data.head()\
        .style\
        .set_caption(f'Сводные сырые данные')\
        .set_table_styles(styler)

In [ ]:
raw_data.shape

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Сырые данные подготовлены, набор данных насчитывает 2471 запись, что не много для хорошего обучения модели. Теперь я хотел бы проверить логику данных по соблюдению технологического процесса, я специально оставил всю информацию по началу и завершению нагрева. Логика проверки такова:<br>
<li>Начало замера температуры должно быть позже начала нагрева;
<li>Последний замер температуры должен быть позже окончания последнего нагрева;
<li>Засыпка материалов должна быть строго в рамках периода нагрева.<br><br>    
Проведем проверку данных
</div>

In [ ]:
raw_data['time_start_check'] = (raw_data['time_start_measure'] >= raw_data['time_start_heat']) & \
                               (raw_data['time_start_bulk']    >= raw_data['time_start_heat']) & \
                               (raw_data['time_start_wire']    >= raw_data['time_start_heat']) & \
                               (raw_data['time_start_bulk']    <= raw_data['time_end_heat'])   & \
                               (raw_data['time_start_wire']    <= raw_data['time_end_heat'])

In [ ]:

raw_data['time_finish_check'] = (raw_data['time_finish_measure'] >= raw_data['time_end_heat'])   & \
                                (raw_data['time_finish_bulk']    <= raw_data['time_end_heat'])   & \
                                (raw_data['time_finish_wire']    <= raw_data['time_end_heat'])   & \
                                (raw_data['time_finish_bulk']    >= raw_data['time_start_heat']) & \
                                (raw_data['time_finish_wire']    >= raw_data['time_start_heat'])

In [ ]:
raw_data.query('time_start_check == False')['time_start_check'].count()

In [ ]:
raw_data.query('time_finish_check == False')['time_finish_check'].count()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Как видно из результатов много несоответствий, да часть из них связана с отсутствием информации по замере температуры. Необходимо удалить эти данные, так же надо будет удалить данные о времени, теперь они больше не нужны
</div>

In [ ]:
raw_data = raw_data.query('time_start_check == True & time_finish_check == True')

In [ ]:
raw_data = raw_data.dropna()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Так же проверю чтобы в финальный датасет не попали данные без информации по загрузке материалов
</div>

In [ ]:
len(raw_data.assign(check = (raw_data.loc[:, 'wire_1':'wire_9']).sum(axis = 1)).query('check == 0'))

In [ ]:
len(raw_data.assign(check = (raw_data.loc[:, 'bulk_1':'bulk_15']).sum(axis = 1)).query('check == 0'))

In [ ]:
set_columns = list(raw_data.filter(like = 'bulk_').columns) + list(raw_data.filter(like = 'wire_').columns)
len(raw_data.assign(check = (raw_data[set_columns]).sum(axis = 1)).query('check == 0'))

In [ ]:
clear_data = raw_data.drop(raw_data.filter(like = 'time').columns.difference(['total_time']), axis = 1)

In [ ]:
clear_data.info()

In [ ]:
clear_data.head()\
          .style\
          .set_caption(f'Сводные чистые данные')\
          .format('{:.0f}')\
          .set_table_styles(styler)

In [ ]:
clear_data.shape[0], len(data_temp["key"].unique())

In [ ]:
'{:.0%}'.format(1 - clear_data.shape[0] / len(data_temp["key"].unique()))

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Таблица подготовлена, в остатке 1982 записей, потеря данных составляет 38% это просто ужасный результат для такого маленького датасета, на этом этапе я бы точно вернул данные тому, кто их формировал
</div>

### Признаки

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
На этом этапе хочу определить какие признаки необходимо оставить для создания модели, т.к. считаю что некоторые из них не информативны. Так например:<br><br>
<li>key — номер партии, уже не имеет значения, ориентир будет индекс;
<li>wire_5 — имеет всего одну запись;
<li>bulk_8 — имеет всего одну запись;<br><br>
Но предварительно посмотрим на тепловую карту корреляции признаков
</div>

In [ ]:
sns.set(rc={'figure.figsize':(15,15)})
sns.heatmap(clear_data.drop('key', axis = 1).corr())

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Как видно на карте <b>wire_5</b> вообще не имеет значений после предобработки только 0, сильная корреляция между показателями <b>wire_8</b> и <b>bulk_9</b>, оставлю <b>bulk_9</b>. Так же, ожидаемый результат по сильной корреляции между <b>total_time</b> и <b>total_power</b>, оставлю из них <b>total_power</b>
</div>

In [ ]:
# версия 2
final_data = clear_data.drop(['key', 'wire_5', 'wire_8', 'bulk_8', 'total_time'], axis = 1).reset_index(drop=True)

In [ ]:
final_data.head()\
          .style\
          .set_caption(f'Сводные чистые данные')\
          .format('{:.0f}')\
          .set_table_styles(styler)

<div style="background-color:gray; border:solid #363636 2px; padding: 20px">      
<span style="color:white"><b>Выводы:</b></span><br><br>
<span style="color:white">    
Данные были загружен и проанализированы, приведены в соответствие и очищены от выбросов, аномалий и пропусков. Необходимо отметить:<br>
<li>Качество предоставленных данных низкого качества;
<li>В данных много пропусков там, где их не должно быть;
<li>Существуют аномалии не в большом количестве, но возможно это мое непонимание технологического процесса;
<li>Большая проблема с временными тегами данных которые нарушают логику технологического процесса, либо опять я не до конца понимаю технологию производства;
<li>По-хорошему такие данные необходимо вернуть на доработку, возможно были допущены ошибки при выгрузке данных, или проконсультироваться с технологами по поводу корректности информации в этих датасетах;
<li>Данные готовы к подготовке и обучению модели.
</span>
</div>

## Подготовка модели

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
    
Основной задачей текущего проекта является прогноз температуры на основании фактических данных. Необходимо подготовить модель, которая способна с высокой долей вероятности прогнозировать конечную температур, что косвенно определяет затраченную  электроэнергию. Целевой признак в моей задаче количественный, который находится в признаках  <b><i>'temperature_finish'</i></b>, соответственно буду использовать модель для регрессии <b>DecisionTreeRegressor, RandomForestRegressor, ExtraTreesRegressor, LGBMRegressor, Lasso, LinearRegression</b> в рамках предсказания, где целевой признак температура. Подбор параметров буду производить с помощью <b>GridSearchCV</b>. Выбор модели на основании метрики с предельно наименьшим значением <b>MAE</b>.
</div>

### Модели с базовыми параметрами

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
    
Перебирать все модели с параметрами я буду с помощью <b>GridSearchCV</b>, для этого подготовлю два списка с моделями <b><i>regressors</i></b> и с гиперпараметрами для этих моделей <b><i>parameters</i></b>. Для начала я решил проверить все модели на базовых гиперпараметрах, чтобы в дальнейшем можно было увидеть динамику изменения качества моделей, для оценки качества моделей выбираю 5 фолдов для кросс-валидации в настройках Grid
</div>

In [ ]:
features = final_data.drop('temperature_finish', axis = 1)
target   = final_data['temperature_finish']

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size = 0.25, 
                                                                              random_state = RANDOM)

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train)    
features_train = scaler.transform(features_train)

In [ ]:
regressors = [LinearRegression(),
              Lasso(random_state = RANDOM),
              DecisionTreeRegressor(random_state = RANDOM),
              ExtraTreesRegressor(random_state = RANDOM),
              RandomForestRegressor(random_state = RANDOM)]

In [ ]:
parameters = [{}] * len(regressors)

In [ ]:
%%time
scores = []

for i in notebook.tqdm(range(len(regressors))):
    
    grid_model = GridSearchCV(estimator  = regressors[i], 
                              param_grid = parameters[i],
                              scoring    = 'neg_mean_absolute_error',
                              n_jobs     = -1,
                              cv         = 5)
    
    grid_model.fit(features_train, target_train)        
    time_score    = grid_model.cv_results_['mean_test_score']   
    mean_fit_time = grid_model.cv_results_['mean_fit_time'][np.where(time_score == time_score.max())[0][0]]
    
    scores.append([' '.join(re.sub(r'([A-Z])', r' \1', str(regressors[i]).split('(')[0]).split()), 
                   grid_model.best_score_, 
                   mean_fit_time,
                   grid_model.best_params_])

In [ ]:
scores_base = scores

In [ ]:
pd.DataFrame(data    = scores_base, 
             columns = ['model', 'mae', 'mean_fit_time', 'parameters'])\
            .sort_values('mae', ascending = False)\
            .reset_index(drop = True)\
            .style\
            .format({'mae':'{:.2f}', 'mean_fit_time' : '{:,.2f}s'})\
            .set_caption(f'Результаты моделей на базовых параметрах',)\
            .set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Отличные результаты на базовых настройках показывают почти все модели, кроме <b>LinearRegression</b>, метрика которой выходит за допустимые значения, лучшую метрику показывает <b>RandomForestRegressor</b>
</div>

### Подбор гиперпараметров

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
    
Далее буду перебрать параметры для моделей и на основании показателей качества моделей выберу лучшую
</div>

In [ ]:
parameters = [{},
              
              {'selection'    : ['cyclic', 'random'],
               'max_iter'     : list(range(100,1000,100))}, 
              
              {'criterion'    : ['friedman_mse', 'poisson'],
               'max_depth'    : list(range(1,10))},
              
              {'n_estimators' : list(range(100,201,10)),
               'max_depth'    : list(range(10,21))},
              
              {'n_estimators' : list(range(100,201,10)),
               'max_depth'    : list(range(10,21))}]

In [ ]:
# результаты моделирования
scores_param = \
[['Linear Regression', -6.246259284306169, 0.006002505620320638, {}],
 ['Lasso',  -6.516956794153191,  0.004667997360229492,  {'max_iter': 100, 'selection': 'cyclic'}],
 ['Decision Tree Regressor',  -7.225941128978035,  0.01299889882405599,  {'criterion': 'friedman_mse', 'max_depth': 4}],
 ['Extra Trees Regressor',  -6.23265048642732,  2.9946855703989663,  {'max_depth': 12, 'n_estimators': 150}],
 ['Random Forest Regressor',  -6.11509833074296,  3.77642289797465,  {'max_depth': 16, 'n_estimators': 100}]]

In [ ]:
pd.DataFrame(data    = scores_param, 
             columns = ['model', 'mae', 'mean_fit_time', 'parameters'])\
            .sort_values('mae', ascending = False)\
            .reset_index(drop = True)\
            .style\
            .format({'mae':'{:.2f}', 'mean_fit_time' : '{:,.2f}s'})\
            .set_caption(f'Результаты поиска лучшей модели с подбором гиперпараметров',)\
            .set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
При подборе гиперпараметров, модели так же показали отличный результат, немного улучшив показатели метрики, разница не такая большая как хотелось бы, на этом этапе результат с применением гиперпараметров моделей, лучшую метрику так же показывает <b>RandomForestRegressor</b>, правда уступая всем моделям по скорости работы алгоритма
</div>

### Модель LightGBM

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Отдельно подготовлю и обучу модель <b>LGBMRegressor</b>, после этого сделаю свод, проанализирую результаты и выберу модель
</div>

In [ ]:
lgb_model = LGBMRegressor(random_state = RANDOM)

In [ ]:
lgb_parameters = {
    'objective'        : ['regression'],
    'metric'           : ['neg_mean_absolute_error'],
    'boosting_type'    : ['gbdt'],
    'num_leaves'       : [10,20,30],
    'learning_rate'    : [0.01],
    'max_depth'        : [5,10,15],
    'n_estimators'     : [5,10,15],
    'num_iterations'   : [500],
    'min_child_samples': [20,30,40]
                 }

In [ ]:
# результаты моделирования
scores_lgb = [['LGBM Regressor', -6.028950006660569, 1.758005936940511,
               {'boosting_type': 'gbdt',
                'learning_rate': 0.01,
                'max_depth': 10,
                'metric': 'neg_mean_absolute_error',
                'min_child_samples': 30,
                'n_estimators': 5,
                'num_iterations': 500,
                'num_leaves': 20,
                'objective': 'regression'}]]

In [ ]:
pd.DataFrame(data    = scores_lgb, 
             columns = ['model', 'mae', 'mean_fit_time', 'parameters'])\
            .sort_values('mae', ascending = False)\
            .style\
            .format({'mae':'{:.2f}', 'mean_fit_time' : '{:,.2f}s'})\
            .set_caption(f'Результаты поиска лучшей модели LightGBM с подбором гиперпараметров',)\
            .set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Модель <b>LGBMRegressor</b> показала лучший результат по качеству модели, не уступая сильно моделям по скорости работы алгоритма. Посмотрим на результаты всех этапов, сделаем выводы и выберем лучшую модель для тестирования
</div>

In [ ]:
data_info = pd.DataFrame(data = np.array(scores_base)[:,:3], 
                          columns = ['model', 'mae_base', 'time_base']) \
                                                                         \
                                .append(pd.DataFrame(data = np.array(scores_lgb)[:,:3],
                                                     columns = ['model', 'mae_base', 'time_base'])) \
                                                                                                     \
                                .merge(pd.DataFrame( data = np.array(scores_param)[:,:3], 
                                                     columns = ['model', 'mae_param', 'time_param']) \
                                                                                                      \
                                                .append(pd.DataFrame(data = np.array(scores_lgb)[:,:3], 
                                                     columns = ['model', 'mae_param', 'time_param'])), 
                                                                   on = 'model', 
                                                                   how = 'outer')

data_info.loc[5][['mae_base', 'time_base']] = 0

In [ ]:
data_info.sort_values(['mae_param', 'time_param'], ascending = [False, True]).reset_index(drop = True) \
                                .style\
                                .format({'mae_base':'{:.2f}', 
                                         'time_base' : '{:,.2f}s',
                                         'mae_param':'{:.2f}', 
                                         'time_param' : '{:,.2f}s'})\
                                .set_caption(f'Результаты этапов обучения моделей',)\
                                .set_table_styles(styler)

<div style="background-color:gray; border:solid #363636 2px; padding: 20px">    
<span style="color:white"><b>Выводы:</b></span><br>   
<span style="color:white">
Как видно из сводных данных выше, лучшие показатели по качеству предсказания показывает модель <b>LGBMRegressor</b>. На данном этапе мы определяем лучшую модель по критериям время/качество. Модель <b>Random Forest Classifier</b>, демонстрирует схожие результаты по качеству моделирования, ее я тоже перенесу на следующий этап, но ужен не в качестве выбранной модели а для чтобы посмотреть как она ведет себя на тестовых данных, так как данная модель показала результаты почти аналогичные выбранной модели.
</span>
</div>

## Тестирование лучшей модели

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Тестирование модели буду проводить на тестовой выборке для выбранной модели на предыдущем этапе
</div>

In [ ]:
features_test = scaler.transform(features_test)

### Тестирование модели Random Forest Regressor (вне проекта)

In [ ]:
final_model_RF = RandomForestRegressor(**scores_param[4][3])

In [ ]:
%%time
start = time()        
final_model_RF.fit(features_train, target_train)     
train_time_RF = time() - start 

In [ ]:
final_pred_RF  = final_model_RF.predict(features_test)
final_score_RF = mean_absolute_error(target_test, final_pred_RF)

In [ ]:
print(f'Показатели MAE на тестовой выборке = {final_score_RF:.2}')

### Тестирование модели LGBM Regressor

In [ ]:
final_model_LGB = LGBMRegressor(**scores_lgb[0][3])

In [ ]:
%%time
start = time()        
final_model_LGB.fit(features_train, target_train)     
train_time_LGB = time() - start 

In [ ]:
final_pred_LGB  = final_model_LGB.predict(features_test)
final_score_LGB = mean_absolute_error(target_test, final_pred_LGB)

In [ ]:
print(f'Показатели MAE на тестовой выборке = {final_score_LGB:.2}')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
И так все этапы пройдены, необходимо посмотреть на результаты всех расчетов
</div>

In [ ]:
final_scores = [['Random Forest Regressor', -final_score_RF, train_time_RF],
                ['LGBM Regressor', -final_score_LGB, train_time_LGB,]]

In [ ]:
data_info \
    .merge(pd.DataFrame(final_scores, columns = ['model', 'mae_final', 'time_final']),
                           on = 'model', how = 'outer') \
                            .sort_values(['mae_final', 'time_final'], ascending = [False, True]).reset_index(drop = True) \
                            .fillna(0).style\
                            .format({'mae_base'  : '{:.2f}', 
                                     'time_base'  : '{:,.2f}s',
                                     'mae_param' : '{:.2f}', 
                                     'time_param' : '{:,.2f}s',
                                     'mae_final' : '{:.2f}',
                                     'time_final' : '{:.2f}s'})\
                            .set_caption(f'Финальные результаты обучения моделей',)\
                            .set_table_styles(styler)

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
В сводной таблице представлены данные за все пройденные этапы, как уже было сказано ранее была выбрана модель <b>LGBM Regressor</b>, как основное решение, которая показала отличный результат на тестовой выборке, модель имеет качество метрики <b>MAE 6,08</b>, задание по подготовке модели выполнено. На тестовой выборке модель <b>Random Forest Regressor</b> модель просела по качеству предсказания значительнее чем основная модель. Далее хотелось бы визуально представить результаты предсказания модели и оценить ее качественные характеристики.<br><br>
Ниже представлена информация, какие признаки внесли больший вклад в обучении модели, здесь можно уже говорить о том что возможно стоило очистить данные в большей степени, Основными фичами оказались общая затраченная энергия и температура на старте,
</div>

In [ ]:
feature_importances = pd.DataFrame(sorted(zip(final_model_LGB.feature_importances_, features.columns)), 
                                   columns = ['Importances','Feature'])

plt.figure(figsize = (10, 10))
sns.set_style('whitegrid')
sns.barplot(x = 'Importances', 
            y = 'Feature', 
            data = feature_importances.sort_values(by = 'Importances', ascending = False),
            palette = 'bone')
plt.title('LGBM Regressor Features Importances', fontsize = 16)
plt.show()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">     
Как и было описано ранее в плане работы, я хотел провести несколько итераций подготовки и обучения моделей, чтобы можно было увидеть качественное изменение метрики в рамках данного проекта. Как видно, модели с самого начала начали справляться с поставленной задачей, но необходимо было улучшить показатели, для стабильности результата на тестовой выборке, для этого были подобраны и применены гиперпараметры моделей для поиска лучшего результата, что в конечном итоге повлияло на отличный результат в финале. На тестовом наборе модели показали результат ниже чем на тренировочном наборе, но эти изменения минимальны, результат стабильный и демонстрирует, что модель хорошо обучилась и разбирается в данных (не переобучена и не до обучена). Соответственно можно сделать выводы, что на стадии предобработки и очистки данных были сделаны корректные изменения
</div>

### Сравнение распределения предсказаний моделей

In [ ]:
# Подготовлю данные в табличном виде для визуализации
data_info = pd.DataFrame(np.array(final_pred_LGB)) \
                    .assign(model = 'LGBM Regressor', sample = 'predict') \
                    .append(pd.DataFrame(np.array(target_test)).assign(model = 'LGBM Regressor', sample = 'true'))

data_info = data_info.append(pd.DataFrame(np.array(final_pred_RF)) \
                    .assign(model = 'Random Forest Regressor', sample = 'predict') \
                    .append(pd.DataFrame(np.array(target_test)).assign(model = 'Random Forest Regressor', sample = 'true'))) \
                    .rename(columns = {0: 'values'})

In [ ]:
# Проверка на корректную сборку таблицы
target_test.shape[0] * 4 - data_info.shape[0]

In [ ]:
plt.figure(figsize=(15, 9))
sns.set_style('whitegrid')

for i, model in enumerate(data_info['model'].unique()):
    
    plt.subplot(1, 2, i + 1)   
    sns.violinplot(x = 'model', 
                   y = 'values', 
                   hue = 'sample',
                   data = data_info[data_info['model'] == model],
                   palette    = 'Reds',
                   saturation = 0.5, 
                   linewidth  = 3,
                   edgecolor  = '#444444',
                   split = True)
    
    plt.title(f'{model}', fontsize = 16);
    plt.ylabel('Temperature', fontsize = 18)
    plt.xlabel('')
    plt.xticks(range(1),[''])
    plt.legend(fontsize = 12)
    
plt.show()

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
На графике видны качества предсказания этих моделей, модели <b>LGBM Regressor</b> и <b>Decision Tree Regressor</b> имеют схожие распределения предсказаний. Но все равно не дотягивают до целевого признака, распределение сильно усредняет значения предсказания, особенно плохо модель работает с верхним и нижнем уровнем температуры, я могу это связать с тем что в целевом наборе, данных для этих диапазонов не достаточно для нормально обучения модели, соответственно не достаточно обучена на этих уровнях. Далее я хотел бы проверить эти показатели на адекватность, для того чтобы убедиться не сильно ли модели усредняют данные по моделированию.
</div>

### Проверка на адекватность

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">    
Окончательно проверю модель на адекватность с помощю <b>DummyRegressor</b>
</div>

In [ ]:
dummy_model  = DummyRegressor()

dummy_model.fit(features_train, target_train) 

dummy_pred   = dummy_model.predict(features_test)

dummy_result = mean_absolute_error(target_test, dummy_pred)

print(f'показатель MAE на модели DummyRegressor: {dummy_result:.2f}')

<div style="background-color:#f3f3f3; border:solid #363636 2px; padding: 20px">
Показатель MAE на модели <b>Dummy Regressor = 7,82</b> это хуже чем на модели <b>LGBM Regressor</b> на 29%, отклонения значительные но не критичные, тем более что даже эта модель уложилась в диапазон требуемого качества, возможно это связано со спецификой самого датасета и температурного диапазона технологического процесса
</div>

## Выводы

<div style="background-color:#424B54; border:solid #373d43 2px; padding: 20px">    
<span style="color:#EBEBEB"><b>Выводы:</b></span><br><br>   
<span style="color:#EBEBEB">
Перед началом работ была описана проблематика работы и поставлена задача, чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Для этого необходимо построить модель, которая предскажет температуру стали. Так же заказчик озвучил требуемые характеристики качества модели со значением метрики качества <b>MAE</b> не более <b>8.7</b>.<br><br>
<li>Предоставленные данные были проанализированы. В наборах данных было обнаружено большое количество артефактов в виде выбросов и аномальных значений, так же присутствовали проблемы, при анализе которых выявлены нарушения технологического процесса, что явно говорит о нарушении выгрузки данных. Данные по-хорошему необходимо отправить на проверку и повторную выгрузку;
<li>Прим моделировании стояла задача регрессии, для этого необходимо было подготовить несколько моделей регрессии. Для этого проекта были выбраны LinearRegression, Lasso, DecisionTreeRegressor, ExtraTreesRegressor, RandomForestRegressor, LGBM Regressor;
<li>Далее последовательно в были обучены и проверены модели, из которой бала выбрана модель с лучшими показателями метрики качества;
<li>На тестовых данных модель показала требуемый результат, и стабильную работу на незнакомых данных, что подтвердило ее высокое качество метрики <b>MAE 6.08</b>;
<li>Задача по подготовке модели была выполнена.<br><br>
<span style="color:#EBEBEB"><b>Итог выбора:</b></span><br><br>    
Лучшая модель LGBM Regressor с параметрами:<br>
<li>boosting_type: 'gbdt',    
<li>learning_rate: 0.01,    
<li>max_depth: 10,    
<li>metric: 'neg_mean_absolute_error',    
<li>min_child_samples: 30,    
<li>n_estimators: 5,    
<li>num_iterations: 500,    
<li>num_leaves: 20,    
<li>objective: 'regression'    
</span>
</div>